## CIFAR10图像分类
Pytorch、两种方法（CNN和ResNet）

In [ ]:
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

fullset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=False, transform=transform)
train_size = int(0.9 * len(fullset))
test_size = len(fullset) - train_size
trainset, validset = torch.utils.data.random_split(fullset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

validloader = torch.utils.data.DataLoader(validset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
                                       download=False, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

如使用数据增强，则采用下一cell的dataloader。

In [ ]:
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

fullset = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=False, transform=transform_train) #训练数据集

train_size = int(0.9 * len(fullset))
test_size = len(fullset) - train_size
trainset, validset = torch.utils.data.random_split(fullset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

validloader = torch.utils.data.DataLoader(validset, batch_size=128,
                                          shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./datasets', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

In [ ]:
print(len(trainloader),len(validloader),len(testloader))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 确认我们的电脑支持CUDA，然后显示CUDA信息：

print(device)

## 1. 两层CNN

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [ ]:
print(net)

In [ ]:
def evaluate(net,inputs,labels):
    with torch.no_grad():
        outputs = net(inputs)
        loss = criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        loss_acc = (predicted == labels).sum().item()
        loss_acc /= len(labels)
        
    return loss,loss_acc

In [ ]:
for epoch in range(50):  # 多批次循环
    train_loss = 0.
    train_acc = 0.
    
    valid_loss = 0.
    valid_acc = 0.
    count1=0
    count2=0
    for i, data in enumerate(trainloader, 0):
        # 获取输入
        count1+=1
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
        # 梯度置0
        optimizer.zero_grad()

        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 打印状态信息
        t_loss,t_acc = evaluate(net,inputs,labels)
        
        train_loss += t_loss
        
        train_acc += t_acc
        
        
    train_loss/=count1
    train_acc/=count1
    
    for data in validloader:
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
            
        v_loss,v_acc = evaluate(net,inputs,labels)
        
        valid_loss += v_loss
        
        valid_acc += v_acc
        
        count2+=1
        
    valid_loss/=count2
    valid_acc/=count2
    
    print('epoch:%d  train_loss:%.4f  train_acc:%.4f  valid_loss:%.4f  valid_acc:%.4f'%((epoch+1),train_loss,train_acc,valid_loss,valid_acc))
    if ((epoch+1)%5==0):
        torch.save(net, './results/lr0.005/model%d_t_acc%.3f_v_acc%.3f.pkl'%((epoch+1),train_acc,valid_acc)) 
    
print('Finished Training')

In [ ]:
net = torch.load("results/lr0.005/model15_t_acc0.758_v_acc0.648.pkl")

In [ ]:
count=0
test_loss=0.
test_acc=0.

for data in testloader:
    count+=1
    inputs, labels = data
        
    inputs, labels = inputs.to(device), labels.to(device)
            
    t_loss,t_acc = evaluate(net,inputs,labels)
        
    test_loss += t_loss
        
    test_acc += t_acc
    
test_loss/=count
test_acc/=count
print("train_loss:%.4f  train_acc:%.4f"%(test_loss,test_acc))

## 2. RESNET 18 

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)


In [ ]:
# 超参数设置
EPOCH = 100   #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 128      #批处理尺寸(batch_size)
LR = 0.01        #学习率

# 模型定义-ResNet
net = ResNet18().to(device)

# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

In [ ]:
print(net)

In [ ]:
for epoch in range(pre_epoch, EPOCH):  # 多批次循环
    train_loss = 0.
    train_acc = 0.
    
    valid_loss = 0.
    valid_acc = 0.
    count1=0
    count2=0
 
    for i, data in enumerate(trainloader, 0):
        # 获取输入
        count1+=1
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
        # 梯度置0
        optimizer.zero_grad()

        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 打印状态信息
        t_loss,t_acc = evaluate(net,inputs,labels)
        
        train_loss += t_loss
        
        train_acc += t_acc
        
        
    train_loss/=count1
    train_acc/=count1
    
    for data in validloader:
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)
            
        v_loss,v_acc = evaluate(net,inputs,labels)
        
        valid_loss += v_loss
        
        valid_acc += v_acc
        
        count2+=1
        
    valid_loss/=count2
    valid_acc/=count2
    
    print('epoch:%d  train_loss:%.4f  train_acc:%.4f  valid_loss:%.4f  valid_acc:%.4f'%((epoch+1),train_loss,train_acc,valid_loss,valid_acc))
    with open("./resnet/log.txt", "w") as f:
        f.write('epoch:%d  train_loss:%.4f  train_acc:%.4f  valid_loss:%.4f  valid_acc:%.4f'%((epoch+1),train_loss,train_acc,valid_loss,valid_acc))
    if ((epoch+1)%5==0):
        torch.save(net, './resnet/zengqiang2/model%d_t_acc%.3f_v_acc%.3f.pkl'%((epoch+1),train_acc,valid_acc)) 
    
print('Finished Training')

In [ ]:
net = torch.load("resnet/zengqiang2/model85_t_acc0.999_v_acc0.912.pkl")

In [ ]:
count=0
test_loss=0.
test_acc=0.

for data in testloader:
    count+=1
    inputs, labels = data
        
    inputs, labels = inputs.to(device), labels.to(device)
            
    t_loss,t_acc = evaluate(net,inputs,labels)
        
    test_loss += t_loss
        
    test_acc += t_acc
    
test_loss/=count
test_acc/=count
print("test_loss:%.4f  test_acc:%.4f"%(test_loss,test_acc))

In [ ]:
#resnet 18网络框架参考 https://blog.csdn.net/weixin_41391619/article/details/104882938